In [ ]:

# Generalized Code for Basic LSTM

# 1. Importing Libraries
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# 2. Loading Dataset Function
def load_dataset(file_path):
    return pd.read_csv(file_path)

# 3. Preprocessing Function
def preprocess_data(df, target_col, look_back=1):
    df = df.copy()
    df.columns = df.columns.str.lower()
    
    X, y = [], []
    for i in range(len(df) - look_back - 1):
        X.append(df.iloc[i:(i+look_back), :].values)
        y.append(df[target_col].iloc[i + look_back])
    
    X, y = np.array(X), np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test

# 4. Model Training and Prediction Function
def train_and_predict(X_train, y_train):
    model = Sequential()
    model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    return model, y_train_pred, y_test_pred

# 5. Evaluation Function
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, predicted)
    return mae, mse, rmse, r2

# 6. Main Execution
if __name__ == "__main__":
    df = load_dataset('path_to_your_dataset.csv')
    X_train, X_test, y_train, y_test = preprocess_data(df, target_col='your_target_column', look_back=3)
    model, y_train_pred, y_test_pred = train_and_predict(X_train, y_train)
    
    train_mae, train_mse, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
    test_mae, test_mse, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(f'Train set evaluation:\nMAE={train_mae}, MSE={train_mse}, RMSE={train_rmse}, R2={train_r2}')
    print(f'Test set evaluation:\nMAE={test_mae}, MSE={test_mse}, RMSE={test_rmse}, R2={test_r2}')
